# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,beyneu,100,KZ,1583983160,41,45.32,55.20,42.04,12.33
1,1,wladyslawowo,20,PL,1583983161,80,54.79,18.40,42.01,10.29
2,2,vaini,90,TO,1583983162,100,-21.20,-175.20,78.80,6.93
3,3,olafsvik,100,IS,1583983163,78,64.89,-23.71,27.10,24.99
4,4,havre-saint-pierre,1,CA,1583983164,85,50.23,-63.60,12.20,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_df[["Lat", "Lng"]]
weights = city_df["Humidity"]

In [8]:
max_weight = weights.max()

In [9]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=max_weight,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
city_df_fit = city_df.loc[((city_df["Max Temp"] > 70.0) & (city_df["Max Temp"] < 80.0)) 
            & (city_df["Wind Speed"] < 10.0) & (city_df["Cloudiness"] == 0)]

hotel_df = city_df_fit.dropna()

In [11]:
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
19,19,paithan,0,IN,1583983184,38,19.48,75.38,73.40,2.24
25,25,nioro,0,GM,1583983190,17,13.35,-15.75,73.72,3.58
156,156,nouakchott,0,MR,1583983309,30,18.09,-15.98,71.60,8.05
220,220,maxixe,0,MZ,1583983417,76,-23.86,35.35,76.68,6.42
281,281,sao filipe,0,CV,1583983118,62,14.90,-24.50,74.28,9.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# Add column Hotel Name to the Dataframe
hotel_df["Hotel Name"] = ""

# Find the closest hotel of each row of the DataFrame

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

for index,row in hotel_df.iterrows():
    coord = f"{row['Lat']},{row['Lng']}"
    params['location'] = coord
    
    # Call the API with the parameters
    response = requests.get(base_url,params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except:
        print(f"Unable to retrieve hotel for city {row['City']}")

Unable to retrieve hotel for city nioro
Unable to retrieve hotel for city bafoulabe


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))